# 706 Assignment 1

# Part 1 -- remaining part

#### ✅ Now try to play with the parameters and watch the histgram change

In [42]:
# Coin Toss Simulator

import random, numpy as np, matplotlib.pyplot as plt, statistics
import ipywidgets as widgets


# core simulation
def run_simulation(repetition: int, sample_size: int):
    results = []
    for i in range(repetition):
        heads = 0
        for j in range(sample_size):
            if random.randint(0, 1) == 1:
                heads += 1
        results.append(heads / sample_size * 100.0)  # percentage scale
    return results

# plot
def show_results(results, sample_size: int, bins: int):
    mean_heads = statistics.mean(results)
    std_heads = statistics.stdev(results) if len(results) > 1 else float("nan")  

    # theoretical moments
    mean_theo = 100 * 0.5  # p scale by percentage
    std_theo  = 100 * np.sqrt(0.5 * 0.5 / sample_size)  # sqrt(p*(1-p)/n) scale by percentage

    # Plot
    fig, ax = plt.subplots(figsize=(8, 5), dpi=120)  #8 inches wide by 5 inches tall; 120 dots (pixels) per inch
    ax.hist(results, bins=bins, density=True)  # here using density not frequency because I want to also show the normal dist curve on the plot
    x = np.linspace(max(0, mean_theo - 6*std_theo), min(100, mean_theo + 6*std_theo), 400)
    # Builds 400 evenly spaced x-values covering ~±6σ around the mean (which contains ~99.999% of the normal mass), clipped to the valid percent range [0, 100].
    y = 1/(std_theo*np.sqrt(2*np.pi)) * np.exp(-0.5*((x-mean_theo)/std_theo)**2) # normal dist. pdf 
    ax.plot(x, y, linewidth=2)
    ax.axvline(mean_heads, color='red',linestyle='--', linewidth=1.5, label=f"Sample μ: {mean_heads:.2f}%")
    ax.axvline(mean_theo, color='yellow', linestyle=':',  linewidth=1.5, label=f"Theoretical μ: {mean_theo:.2f}%")
    ax.grid(alpha=0.3)  #alpha is opacity: 0 = fully transparent, 1 = fully opaque
    ax.set_xlabel('% of Heads')
    ax.set_ylabel('Density')
    ax.set_title(f'Distribution of %Heads | n={sample_size} per experiment, reps={len(results)}')
    ax.legend(frameon=False, loc='upper left')
    plt.show()

    import ipywidgets as widgets
    from ipywidgets import VBox, Box, HTML, Layout
    # VBox: a stacker. It places things top to bottom (like stacking books in a column).
    # Box: a container you can customize. You decide if items go left-to-right, wrap to the next line, spacing, alignment, etc. (Think “flexible shelf”.)
    # HTML: a fancy label. It shows text with basic formatting (bold, line breaks). Easier than drawing text on the plot.
    # Layout: the style card. You use it to give any widget size, padding, borders, rounded corners, spacing, and flex settings.
    def metric_card(title, value):
        return VBox(
            [HTML(f"<b>{title}</b>"), HTML(value)],
            layout=Layout(
                padding='10px 14px',   #the space inside the card, between the border and the text.top & bottom; left & right;top, right, bottom, left;
                border='1px solid #ddd', # solid/dashed
                border_radius='9999px', # how rounded the corners are; 0 sharpe, 20px round, 9999px fully round
                width='auto'
            )
        )
    # Stacks two lines vertically:
    # HTML(f"<b>{title}</b>") → the title, bold
    # HTML(value) → the value underneath
    # layout=Layout(...) styles the card:
    # padding='10px 14px' → space inside the card
    # border='1px solid #ddd' → thin light-gray border
    # border_radius='20px' → rounded corners
    # width='auto' → card shrinks/grows to fit its content
    def show_metric_cards(mean_heads, std_heads, mean_theo, std_theo):
        cards = Box(
            [
                metric_card("Sample mean",      f"{mean_heads:.4f}%"),
                metric_card("Sample std",       f"{std_heads:.4f}%"),
                metric_card("Theoretical mean", f"{mean_theo:.2f}%"),
                metric_card("Theoretical std",  f"{std_theo:.4f}%"),
            ],
            layout=Layout(display='flex', flex_flow='row wrap', gap='14px')
        # display='flex' Treat this Box like a flex row/column layout instead of a simple stack.
        # flex_flow='row nowrap' → keep everything on one line (may overflow/squish); flex_flow='column' → stack items top-to-bottom
        # flex_flow : row: left → right; wrap: if there isn’t enough width, move overflow items to the next line
        )
        display(cards)
    show_metric_cards(mean_heads, std_heads, mean_theo, std_theo)


# slider factory
def make_labeled_slider(title, minv, maxv, value, step):
    slider = widgets.IntSlider(
        min=minv, max=maxv, value=value, step=step,
        readout=False, continuous_update=False, layout=widgets.Layout(width='520px')
        # readout=False: hides the slider’s built-in number bubble—we’ll show our own labels instead.
        # continuous_update=False: only fires a value change when the user releases the mouse
        # layout=widgets.Layout(width='520px'): fixes the visual track width to 520 pixels so the UI aligns nicely
    )
    lbl_title = widgets.HTML(f"<b>{title}</b>", layout=widgets.Layout(width='130px'))
    # f"<b>{title}</b>": bolds the passed-in title
    lbl_min   = widgets.HTML(f"<span style='color:#666;'>{minv}</span>", layout=widgets.Layout(width='60px'))
    lbl_cur   = widgets.HTML(f"<span style='font-variant-numeric:tabular-nums;'>{value}</span>",
                             layout=widgets.Layout(width='80px', justify_content='center'))
    # font-variant-numeric: tabular-nums: makes digits monospaced so values don’t “wiggle” as they change
    lbl_max   = widgets.HTML(f"<span style='color:#666;'>{maxv}</span>", layout=widgets.Layout(width='60px'))
    """
    <span> = “start styling this little piece of text.”
    style="..." = the clothes you put on it (color, font, etc.).
    </span> = “stop styling now.” 
    """

    def on_change(change):
        if change['name'] == 'value':
            lbl_cur.value = f"<span style='font-variant-numeric:tabular-nums;'>{change['new']}</span>" # update value with user's change
    slider.observe(on_change, names='value')
    """
    on_change(change): a callback that runs when a widget trait changes
    change is a dict like {'name': 'value', 'old': 1000, 'new': 1200, 'owner': slider, ...}
    font-variant-numeric: tabular-nums makes digits equal-width
    """

    numbers = widgets.HBox([
        lbl_title,
        widgets.HTML("min&nbsp;"), lbl_min,
        widgets.HTML("&nbsp;|&nbsp; current&nbsp;"), lbl_cur,
        widgets.HTML("&nbsp;|&nbsp; max&nbsp;"), lbl_max
    ], layout=widgets.Layout(align_items='center', gap='4px')) # align_items='center' → vertically center all items in the row

    return widgets.VBox([numbers, slider]), slider

#  UI 
rep_row,   rep_slider   = make_labeled_slider("Repetitions", 100, 20000, 1000, 100)
size_row,  size_slider  = make_labeled_slider("Sample size", 10,  5000,  100,  10)
bins_row,  bins_slider  = make_labeled_slider("Bins",        5,   80,    20,   1)

run_btn  = widgets.Button(description="Run simulation", button_style='primary', icon='play')
status   = widgets.HTML("<span style='color:#666;'>Ready</span>")
out      = widgets.Output()
"""
description="Run simulation" → the text shown on the button.
button_style='primary' → applies a built-in color theme. Valid values: '' (default/gray), 'primary', 'success', 'info', 'warning', 'danger'.
icon='play' → adds a small icon to the left of the text (uses Font Awesome names, e.g., 'play', 'refresh', 'check', 'pause').
"""

def on_click(_):
    status.value = "<span style='color:#666;'>Running…</span>"
    with out:
        out.clear_output(wait=True)
        # Clears whatever was shown last time
        # wait=True reduces flicker: it waits to show the new output until it’s ready
        res = run_simulation(rep_slider.value, size_slider.value)
        show_results(res, size_slider.value, bins_slider.value)  #hist
    status.value = "<span style='color:green;'>Done</span>" # After the block completes, change the status to green “Done”

run_btn.on_click(on_click)  # clicking the button calls on_click(...) once

header = widgets.HTML("<h3 style='margin:0'>Coin Toss Simulator</h3>"
                      "<p style='margin:4px 0 10px;color:#666'>Adjust parameters, then click <b>Run simulation</b>.</p>")
"""
<h3> = a level-3 heading (big bold title)
style='margin:0' = CSS that removes the default outside spacing around the heading (no gap above/below)
<p> = a paragraph (normal text)
"""
ui = widgets.VBox([header, rep_row, size_row, bins_row, widgets.HBox([run_btn, status])]) # stacks vertically (top→bottom)
display(ui, out)


Output()

# Part 4

## >>> Create a Dynamic Dashboard

In [41]:

import sys, platform, datetime as dt 
# sys — Python runtime info
# platform — details about the OS, Windows or Mac..
import numpy as np
import pandas as pd
import yfinance as yfin
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, HTML  # makes notebook output look nice

def parse_tickers(text):
    return [t.strip() for t in text.split(",") if t.strip()]
    """
    t.strip() removes spaces in both ends, 'if' keeps only tickers that aren’t empty
    tickers seperate by ',', if there are empty inputs
    limited use, modification required
    """

def parse_weights(text, n):
    try:
        vals = [float(x.strip()) for x in text.split(",") if x.strip()]
        # turn user input weights into float
        if len(vals) != n: # if there the #weights not align with #tickers
            return np.repeat(1.0/n, n)  # using equal weights by default
        w = np.array(vals, dtype=float)
        if (w < 0).any() or w.sum() == 0: # if input weights <0 or sum=0, using equal weights
            return np.repeat(1.0/n, n)
        return w / w.sum()  # if no above problems, normalize the weights (in case sum >1) 
    except Exception:
        return np.repeat(1.0/n, n)   # if anything at all goes wrong (e.g., non-numeric input), return equal weights

def freq_label_to_code(label):
    return {"Daily":"D", "Weekly":"W", "Monthly":"M"}[label]
"""
Turns a label into a Pandas resample code using a dictionary lookup.
Mapping:
"Daily" → "D" (calendar daily frequency)
"Weekly" → "W" (weekly)
"Monthly" → "M" (month-end)
"""
def periods_per_year(freq_code):
    return {"D":252, "W":52, "M":12}[freq_code]
"""
Converts a frequency code to the number of periods per year, useful for annualizing stats:
"D" → 252 (trading days per year)
"W" → 52 (weeks per year)
"M" → 12 (months per year)
"""

def system_info_line():
    return (f"{platform.system()} {platform.release()} | "
            f"Python {platform.python_version()} | "
            f"NumPy {np.__version__} | Pandas {pd.__version__} | yfinance {getattr(yfin, '__version__','N/A')}")
# print computer system information as asked
# Windows;11(release);
# sector names for friendlier display
SECTOR_NAMES = {
    '^GSPE': 'S&P500 Energy SECTOR',
    '^SP500-10': 'S&P500 Energy SECTOR',
    '^SP500-15': 'S&P500 Materials SECTOR',
    '^SP500-20': 'S&P500 Industrials SECTOR',
    '^SP500-40': 'S&P500 Financials SECTOR',
    '^SP500-55': 'S&P500 Utilities SECTOR',
    '^SP500-25': 'S&P500 Consumer Discretionary SECTOR',
    '^SP500-30': 'S&P500 Consumer Staples SECTOR',
    '^SP500-35': 'S&P500 Health Care SECTOR',
    '^SP500-45': 'S&P500 Information Technology SECTOR',
    '^SP500-50': 'S&P500 Communication Services SECTOR',
    '^SP500-60': 'S&P500 Real Estate SECTOR',
}
def pretty_ticker(t):
    t = "" if t is None else str(t).strip() # change to string type
    label = SECTOR_NAMES.get(t, "Unknown sector") # if cannot find, show unknown
    return f"{label} — {t}"  # show sectornames, then show ticker

# labeled slider: min | current | max
def make_labeled_slider(title, minv, maxv, value, step):
    slider = widgets.IntSlider(min=minv, max=maxv, value=value, step=step,
                               readout=False, continuous_update=False,
                               layout=widgets.Layout(width='520px'))
    """
    readout=False: hides the slider’s built-in number, show designed labels instead
    continuous_update=False: only fires a value change when the user releases the mouse
    layout=widgets.Layout(width='520px'): fixes the visual track width to 520 pixels so the UI aligns nicely
    """
    lbl_title = widgets.HTML(f"<b>{title}</b>", layout=widgets.Layout(width='150px'))
    # b for bold, text format
    lbl_min   = widgets.HTML(f"<span style='color:#666;'>{minv}</span>", layout=widgets.Layout(width='60px'))
    
    lbl_cur   = widgets.HTML(f"<span style='font-variant-numeric:tabular-nums;'>{value}</span>",
                             layout=widgets.Layout(width='80px', justify_content='center'))
    lbl_max   = widgets.HTML(f"<span style='color:#666;'>{maxv}</span>", layout=widgets.Layout(width='60px'))
    """
    font-variant-numeric: tabular-nums: makes digits monospaced so values don’t “wiggle” as they change
    <span> = “start styling this little piece of text.”
    style="..." = the clothes you put on it (color, font, etc.).
    </span> = “stop styling now.” 
    """

    def on_change(change):
        if change['name'] == 'value':
            lbl_cur.value = f"<span style='font-variant-numeric:tabular-nums;'>{change['new']}</span>" # update value with user's change
    slider.observe(on_change, names='value')
    """
    on_change(change): a callback that runs when a widget trait changes
    change is a dict like {'name': 'value', 'old': 1000, 'new': 1200, 'owner': slider, ...}
    font-variant-numeric: tabular-nums makes digits equal-width
    """

    row = widgets.VBox([
        widgets.HBox([lbl_title,
                      widgets.HTML("min&nbsp;"), lbl_min,
                      widgets.HTML("&nbsp;|&nbsp; current&nbsp;"), lbl_cur,
                      widgets.HTML("&nbsp;|&nbsp; max&nbsp;"), lbl_max],
                     layout=widgets.Layout(align_items='center', gap='4px')),
        slider
    ])
    return row, slider
    """
    widgets.VBox([...]): a vertical container. It stacks its children top to bottom
    Child 1 is an HBox (a horizontal row of small widgets)
    Child 2 is the slider (likely an IntSlider or FloatSlider created elsewhere)
    widgets.HBox([...], layout=...): a horizontal container. It lays out its children left to right
    &nbsp is a non-breaking space to keep spacing
    A row with: [Title] min [min_value] | current [cur_value] | max [max_value]
    """
    

def show_metric_cards(mean_final, std_final, prob_loss, n_repeat, horizon_days):
    cards = widgets.Box([
        widgets.VBox([widgets.HTML("<b>Mean final value</b>"), widgets.HTML(f"{mean_final:.4f}")],
                     layout=widgets.Layout(padding='10px 14px', border='1px solid #ddd', border_radius='12px')),
        widgets.VBox([widgets.HTML("<b>Std dev final value</b>"), widgets.HTML(f"{std_final:.4f}")],
                     layout=widgets.Layout(padding='10px 14px', border='1px solid #ddd', border_radius='12px')),
        widgets.VBox([widgets.HTML("<b>P(loss &gt; 10%)</b>"), widgets.HTML(f"{prob_loss:.4f}")],
                     layout=widgets.Layout(padding='10px 14px', border='1px solid #ddd', border_radius='12px')),
        widgets.VBox([widgets.HTML("<b>Runs × Horizon</b>"), widgets.HTML(f"{n_repeat} × {horizon_days} days")],
                     layout=widgets.Layout(padding='10px 14px', border='1px solid #ddd', border_radius='12px')),
    ], layout=widgets.Layout(display='flex', flex_flow='row wrap', gap='12px'))
    display(cards)
"""
widgets.Box([...]): A generic container. Here it acts as a flexbox row
Styling per card via layout=widgets.Layout(...):
padding='10px 14px' → space inside the card
border='1px solid #ddd' → thin light-gray border
border_radius='20px' → rounded corners
width='auto' → card shrinks/grows to fit its content
padding: the space inside the card, between the border and the text.top & bottom; left & right;top, right, bottom, left;
border: solid/dashed
border_radius: how rounded the corners are; 0 sharpe, 20px round, 9999px fully round
display='flex' Treat this Box like a flex row/column layout instead of a simple stack.
flex_flow='row nowrap' → keep everything on one line (may overflow/squish); flex_flow='column' → stack items top-to-bottom
flex_flow : row: left → right; wrap: if there isn’t enough width, move overflow items to the next line
"""
  
def run_simulation(tickers, weights_str, start, end, freq_label, horizon_days, n_repeat):
    tickers_list = parse_tickers(tickers)
    if not tickers_list:
        raise ValueError("Please provide at least one ticker.")
    freq_code = freq_label_to_code(freq_label)
    k = periods_per_year(freq_code)

    # prices
    data = yfin.download(tickers_list, start=start, end=end, actions=False, auto_adjust=False, progress=False)
    if 'Adj Close' not in data:
        raise ValueError("Could not fetch Adj Close prices. Check tickers/date range.")
    prices = data['Adj Close'].dropna()

    # resample to get different returns under different frequency 
    if freq_code == "W":
        prices = prices.resample('W-FRI').last().dropna()
    elif freq_code == "M":
        prices = prices.resample('ME').last().dropna()   # <- avoids FutureWarning

    if isinstance(prices, pd.Series):  #if only got a single Series, convert it to a one-column DataFrame
        prices = prices.to_frame()

    # parameters
    logret = np.log(prices).diff().dropna()
    mu  = logret.mean() * k
    cov = logret.cov()  * k
    var = np.diag(cov.values)

    n = prices.shape[1]  # the column number is how many stocks 
    w = parse_weights(weights_str, n)
    S0_indiv = 100.0 * w

    dt = 1/252
    drift = (mu.values - 0.5 * var) * dt   # GBM

    try:  #create a matrix that turns independent normals into correlated shocks
        L = np.linalg.cholesky(cov.values * dt)  #lower-triangular L such that L @ Lᵀ = Σ
    except np.linalg.LinAlgError:
        L = np.linalg.cholesky(cov.values * dt + 1e-12*np.eye(n))
        """
        if numerical issues make cov*dt not quite positive-definite;
        the except block regularizes it by adding a tiny εI (1e-12*np.eye(n)) so the Cholesky succeeds.
        """

    rng = np.random.default_rng()  #random generator
    final_vals = np.zeros(n_repeat, dtype=float)
    
    for rep in range(n_repeat):
        S = S0_indiv.copy()
        for step in range(horizon_days):
            z = rng.standard_normal(n)  
            shock = L @ z  # correlated normal shock
            S *= np.exp(drift + shock)  #GBM
        final_vals[rep] = S.sum()          # or float(S.sum())

    return final_vals, tickers_list, w, mu, cov

#  UI 
default_tickers = "^GSPE,^SP500-40,^SP500-15,^SP500-20,^SP500-55"
default_weights = "0.2,0.2,0.2,0.2,0.2"

tickers_box = widgets.Text(value=default_tickers, description='Tickers:',
                           layout=widgets.Layout(width='800px'))
weights_box = widgets.Text(value=default_weights, description='Weights:',
                           layout=widgets.Layout(width='300px'))

freq_dd = widgets.Dropdown(options=['Daily','Weekly','Monthly'], value='Daily',
                           description='Frequency:', style={'description_width':'initial'})

horizon_dd = widgets.Dropdown(options=[30,120,250,500], value=120,
                              description='Horizon (days):',
                              style={'description_width':'initial'},
                              layout=widgets.Layout(width='220px'))

rep_row,  rep_slider  = make_labeled_slider("Repetitions", 100, 20000, 1000, 100) #min, max, default, step
bins_row, bins_slider = make_labeled_slider("Bins",        10,  100,    30,   2)

run_btn = widgets.Button(description="Run simulation", button_style='primary', icon='play')
status  = widgets.HTML("<span style='color:#666;'>Ready</span>")
out     = widgets.Output()
"""
description="" → the text shown on the button.
button_style='primary' → applies a built-in color theme. Valid values: '' (default/gray), 'primary', 'success', 'info', 'warning', 'danger'.
icon='play' → adds a small icon to the left of the text (uses Font Awesome names, e.g., 'play', 'refresh', 'check', 'pause').
"""
header = widgets.HTML(
    "<h1 style='margin:0'>Correlated GBM Portfolio Simulator</h3>"
    # h1,a level-2 heading (big bold title)
    "<p style='margin:4px 0 10px;color:#666'>"
    "Change parameters, then click <b>Run simulation</b>. "
    "Tickers and weights are comma-separated; weights will be normalized.</p>"
    #style='margin:4px' = CSS that removes the default outside spacing around the heading (no gap above/below)
    #<p> = a paragraph (normal text)
)

controls = widgets.VBox([
    header,
    widgets.HBox([tickers_box, weights_box], layout=widgets.Layout(gap='10px')),
    # A horizontal row that places the Tickers text box and Weights text box side by side
    widgets.HBox([freq_dd, horizon_dd], layout=widgets.Layout(gap='10px')),
    rep_row, bins_row,
    widgets.HBox([run_btn, status], layout=widgets.Layout(gap='10px'))
])

display(controls, out)
# Renders the control panel and, beneath it, the output area. Users interact with the widgets in controls; your code writes results into out
#  callback 
def on_click(_):   # Defines the callback that runs when the button is clicked.
    # _ argument is the click event object; you don’t use it, so _ is a common placeholder
    run_btn.disabled = True # Prevents double-clicks while a simulation is running
    status.value = "<span style='color:#666;'>Running…</span>" # Sets a grey status label so the user knows it’s busy.
    with out:
        out.clear_output(wait=True)  # erases the previous run’s output
        try:
            final_vals, tickers_used, w_used, mu, cov = run_simulation(
                tickers_box.value, weights_box.value,
                start="2020-01-01", end="2024-12-31",
                freq_label=freq_dd.value, horizon_days=horizon_dd.value,
                n_repeat=rep_slider.value
            )
            """
            Calls simulation with the current UI values: tickers, weights, date range, frequency, horizon, and number of repetitions.
            """
            mean_final = final_vals.mean()
            std_final  = final_vals.std(ddof=1)
            prob_loss  = (final_vals < 90).mean()

            # hist
            fig, ax = plt.subplots(figsize=(8,5), dpi=140)
            ax.hist(final_vals, bins=bins_slider.value, edgecolor='black',color='lightgreen')
            ax.set_xlabel(f'Final Portfolio Value (after {horizon_dd.value} trading days)')
            ax.set_ylabel('Frequency')
            ax.set_title(f'Equal-weight Portfolio: Final Values ({rep_slider.value} sims)')
            ax.grid(alpha=0.3) #opaque

            plt.tight_layout()
            plt.subplots_adjust(bottom=0.18)             # more space at bottom, to show the system info 
            fig.text(0.99, 0.03, system_info_line(),     # system info with better spacing
                     ha='right', va='bottom', fontsize=8)
            plt.show()

            # metrics
            show_metric_cards(mean_final, std_final, prob_loss, rep_slider.value, horizon_dd.value)

            # renders metric cards (mean, stdev, P(loss>10%), runs × horizon) under the chart.
            rows = []
            for t, w_i in zip(tickers_used, w_used):
                rows.append(f"<li>{pretty_ticker(t)}: <b>{w_i:.2%}</b></li>")
            tick_list_html = "<ul style='margin:4px 0 8px 18px;'>" + "".join(rows) + "</ul>"

            display(HTML(
                "<div style='color:#444; margin-top:8px;'>"
                f"<b>Portfolio (by sector):</b>{tick_list_html}"
                f"<b>Return frequency used to estimate μ and Σ:</b> {freq_dd.value}"
                "</div>"
            ))
            """
            <ul> ... </ul> Unordered list, a bullet list container
           <li> ... </li> List item, one bullet inside a list
           <div> ... </div> A generic block container. Used to group content for layout or styling. It does not add bullets or numbers.
            """
        except Exception as e:
            print("Error:", e)
        # if anything fails (bad ticker, network hiccup, etc.), print a readable error into out instead of crashing
        finally:
            status.value = "<span style='color:green;'>Done</span>"
            run_btn.disabled = False
            # Updates the status to green “Done”
            # Re-enables the Run button so the user can try again.

run_btn.on_click(on_click)  # registers callback. Now clicking the button triggers everything above.


Output()